In [4]:
import cv2
import mediapipe as mp 
import numpy as np
import math
import time
import requests

In [5]:
def draw_text(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.5, color=(255, 255, 255), thickness=1):
    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)

    
    # Function to draw a filled rectangle
def draw_filled_rectangle(image, start_point, end_point, color):
    cv2.rectangle(image, start_point, end_point, color, cv2.FILLED)

#function to detect falls
def detect_fall(landmarks, image_height):
    # Convert normalized y-coordinates to absolute pixel values
    left_shoulder_y = int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y * image_height)
    left_hip_y = int(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y * image_height)
    right_shoulder_y = int(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y * image_height)
    right_hip_y = int(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y * image_height)
    fall_threshold = 40 
    # Check if the difference between shoulder and hip y-coordinates is less than the threshold
    if left_hip_y - left_shoulder_y < fall_threshold or right_hip_y - right_shoulder_y < fall_threshold:
        return True
    else:
        return False
#function to calculate angle btween 3 points
def calculate_angle(a,b,c):
    a=np.array(a) #first 
    b=np.array(b) #mid
    c=np.array(c) #last
    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*108.0/np.pi)
    if angle >180.0:
        angle=360-angle
    return angle


def calculate_distance(point1, point2):
    return math.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)

def normalize_distance(distance, reference_distance):
    return distance / reference_distance

import time

def distress_detection(landmarks, image, duration=10, print_interval=3):
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_hand = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_hand = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

    # Calculate the distance between the shoulders
    shoulder_distance = calculate_distance(left_shoulder, right_shoulder)

    # Normalize distances by dividing by shoulder distance
    distance_left_hand_shoulder_normalized = normalize_distance(
        calculate_distance(left_hand, left_shoulder),
        shoulder_distance
    )
    distance_right_hand_shoulder_normalized = normalize_distance(
        calculate_distance(right_hand, right_shoulder),
        shoulder_distance
    )

    # Calculate the angle between left elbow, left wrist, and left shoulder
    angle_left_arm = calculate_angle(
        [left_elbow.x, left_elbow.y],
        [left_hand.x, left_hand.y],
        [left_shoulder.x, left_shoulder.y]
    )

    # Calculate the angle between right elbow, right wrist, and right shoulder
    angle_right_arm = calculate_angle(
        [right_elbow.x, right_elbow.y],
        [right_hand.x, right_hand.y],
        [right_shoulder.x, right_shoulder.y]
    )
    distress_flag = False
    # Check if it's time to print
    current_time = time.time()
    if (
        (0.5 < distance_right_hand_shoulder_normalized < 1 and (110 < angle_right_arm < 145 or 60 < angle_left_arm < 100)) or
        (0.2 < distance_left_hand_shoulder_normalized < 0.8 and (110 < angle_right_arm < 145 or 60 < angle_left_arm < 100))
    ):
        distress_detection.last_print_time = current_time

        # Display "warning" text during the countdown
        draw_text(image, "warning", (20, 100), color=(0, 0, 255))

        if not hasattr(distress_detection, 'start_time'):
            distress_detection.start_time = current_time
        elif current_time - distress_detection.start_time > duration:
            countdown = int(duration - (current_time - distress_detection.start_time))

            # Display the countdown
            draw_text(image, str(countdown), (20, 70), color=(255, 0, 0))

            if countdown <= 10:
                # Switch to "danger" text
                draw_text(image, "danger", (20, 100), color=(255, 0, 0))

            if countdown <= 0:
                distress_flag = True

    return distress_flag

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Define your functions (including draw_text, draw_filled_rectangle, detect_fall, calculate_angle, calculate_distance, distress_detection, bending_over)

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Initialize start_time outside the loop
distress_detection.start_time = None

cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.7, min_tracking_confidence=0.7) as pose:
    while cap.isOpened(): 
        ret, frame = cap.read()
        # Recolor image 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detection 
        results = pose.process(image)
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        # Extract landmarks 
        try:
            landmarks = results.pose_landmarks.landmark
            left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
            left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
            distress_detection(landmarks, image)
            
            # Draw filled rectangle as a background
            start_point = (5, 10)
            end_point = (400, 60)
            rectangle_color = (0, 0, 255)  # Blue color
            
            
            # Draw text on the image to display fall state
            if detect_fall(landmarks, image.shape[0]):
                text = "Fallen"
            else:
                left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
                shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
                left_hand = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
                right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
                text=str( angle_left_arm = calculate_angle(
        [left_elbow.x, left_elbow.y],
        [left_hand.x, left_hand.y],
        [left_shoulder.x, left_shoulder.y]
    ))
            draw_text(image, text, (20, 40), color=(255, 255, 255),font_scale=2.0)

                
                
            

        except Exception as e:
            pass 
            
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord("q"): 
            break

cap.release()
cv2.destroyAllWindows()
